# Exotic Derivatives Trading Desk Analysis

**As of 16 May 2025**, we will:
1. Fetch market data  
2. Instantiate each of our four option classes  
3. Price & (where available) compute delta  
4. Summarize hedges for the combined portfolio  


In [1]:
# Install needed packages (run this cell once)
!pip install --quiet yfinance scipy

In [2]:
# Cell 2: imports and discount-factor setup
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

from derivatives import (
    EuropeanCall,
    AmericanPut,
    UpAndInBarrierCall,
    BasketCall
)

# Flat 4% risk-free curve → discount factor function
r = 0.04
discount = lambda T: np.exp(-r * T)


In [8]:
# Cell 2.5: Yield‐curve bootstrapping

import numpy as np
from scipy.interpolate import interp1d

# 1. Market tenors (years) and continuous zero rates (as decimals)
tenors     = np.array([1/12, 3/12, 6/12, 1.0, 3.0, 5.0, 10.0, 15.0])
zero_rates = np.array([
    0.037994,  # 1 m
    0.037990,  # 3 m
    0.038575,  # 6 m
    0.035670,  # 1 y
    0.035185,  # 3 y
    0.038823,  # 5 y
    0.043145,  # 10 y
    0.045320   # 15 y
])

# 2. Discount factors at those tenors
discount_factors = np.exp(-zero_rates * tenors)

# 3. Build a linear zero‐rate interpolator
zero_interp = interp1d(
    tenors,
    zero_rates,
    kind="linear",
    fill_value="extrapolate",
    assume_sorted=True
)

# 4. Override discount(T) to use the interpolated curve
discount = lambda T: np.exp(-zero_interp(T) * T)

# Quick check
print("DF(0.5 y) =", discount(0.5))
print("DF(2 y)   =", discount(2.0))


DF(0.5 y) = 0.9808973137225787
DF(2 y)   = 0.931596963894417


In [9]:
# Cell 3: Market inputs as of 16 May 2025

trade_date = "2025-05-16"
next_day   = (
    datetime.datetime.strptime(trade_date, "%Y-%m-%d")
    + datetime.timedelta(days=1)
).strftime("%Y-%m-%d")

tickers = ["BHP.AX","CBA.AX","WES.AX","CSL.AX","WDS.AX","MQG.AX"]
df = yf.download(tickers, start=trade_date, end=next_day)["Close"]
S0 = df.loc[trade_date].to_dict()

# Placeholder implied vols
vol = {
    "BHP.AX": 0.25,
    "CBA.AX": 0.22,
    "WES.AX": 0.30,
    "CSL.AX": 0.20,
    "WDS.AX": 0.28,
    "MQG.AX": 0.23
}

print("Spot prices:", S0)
print("Volatilities:", vol)
print("Flat r:", r)


[**********************67%*******                ]  4 of 6 completed

[*********************100%***********************]  6 of 6 completed

Spot prices: {'BHP.AX': 39.720001220703125, 'CBA.AX': 169.66000366210938, 'CSL.AX': 241.82000732421875, 'MQG.AX': 207.35000610351562, 'WDS.AX': 21.920000076293945, 'WES.AX': 82.55999755859375}
Volatilities: {'BHP.AX': 0.25, 'CBA.AX': 0.22, 'WES.AX': 0.3, 'CSL.AX': 0.2, 'WDS.AX': 0.28, 'MQG.AX': 0.23}
Flat r: 0.04


In [10]:
# Cell 4: Instantiate & price

# 1) BHP European Call
T1         = (datetime.datetime(2027,9,15) - datetime.datetime(2025,5,16)).days/365
bhp_call   = EuropeanCall(
    S0       = S0["BHP.AX"],
    K        = 0.98 * S0["BHP.AX"],
    T        = T1,
    discount = discount,
    sigma    = vol["BHP.AX"]
)
price_bhp, delta_bhp = bhp_call.price(), bhp_call.delta()

# 2) CBA American Put
T2        = (datetime.datetime(2026,5,15) - datetime.datetime(2025,5,16)).days/365
cba_put   = AmericanPut(
    S0       = S0["CBA.AX"],
    K        = 170.0,
    T        = T2,
    discount = discount,
    sigma    = vol["CBA.AX"]
)
price_cba, delta_cba = cba_put.price(), cba_put.delta()

# 3) WES Up-and-In Barrier Call
wes_bar   = UpAndInBarrierCall(
    S0       = S0["WES.AX"],
    K        = 80.0,
    T        = T1,
    discount = discount,
    sigma    = vol["WES.AX"],
    barrier  = 100.0
)
price_wes, delta_wes = wes_bar.price(), wes_bar.delta()

# 4) Four-Stock Basket Call
T3         = (datetime.datetime(2025,7,17) - datetime.datetime(2025,5,16)).days/365
basket     = BasketCall(
    S0_list    = [S0[t] for t in ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]],
    weights    = [0.10,0.35,0.15,0.40],
    K          = 175.0,
    T          = T3,
    discount   = discount,
    sigma_list = [vol[t] for t in ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]]
)
price_bask, delta_bask = basket.price(), basket.delta()

# Quick print
print(f"BHP Call     → Price: {price_bhp:.2f}, Δ: {delta_bhp:.4f}")
print(f"CBA Put      → Price: {price_cba:.2f}, Δ: {delta_cba:.4f}")
print(f"WES Barrier  → Price: {price_wes:.2f}, Δ: {delta_wes:.4f}")
print(f"Basket Call  → Price: {price_bask:.2f}, Δ: {delta_bask:.4f}")


BHP Call     → Price: 7.86, Δ: 0.6772
CBA Put      → Price: 170.00, Δ: -0.0000
WES Barrier  → Price: 18.35, Δ: 10.6249
Basket Call  → Price: 0.19, Δ: 0.0789


In [11]:
# Cell 5: Build & format DataFrame of Price & Greeks safely

import numpy as np
import pandas as pd

def safe_vega(opt):
    """Call opt.vega(), but return NaN if it fails (e.g. BasketCall)."""
    try:
        return opt.vega()
    except Exception:
        return np.nan

# 1) Build raw Greeks table
df_greeks = pd.DataFrame({
    "Instrument": ["BHP Call", "CBA Put", "WES Barrier", "Basket Call"],
    "Price":      [price_bhp, price_cba, price_wes, price_bask],
    "Delta":      [delta_bhp, delta_cba, delta_wes, delta_bask],
    "Vega":       [
        safe_vega(bhp_call),
        safe_vega(cba_put),
        safe_vega(wes_bar),
        safe_vega(basket)
    ],
    "Theta":      [
        bhp_call.theta(),
        cba_put.theta(),
        wes_bar.theta(),
        basket.theta()
    ]
})

# 2) Round all numeric columns to 2 decimals
df_fmt = df_greeks.copy()
for col in ["Price", "Delta", "Vega", "Theta"]:
    df_fmt[col] = df_fmt[col].round(2)

# 3) Format Price as currency
df_fmt["Price"] = df_fmt["Price"].map("${:,.2f}".format)

# 4) Display
df_fmt


,Instrument,Price,Delta,Vega,Theta
0,BHP Call,$7.86,0.68,21.78,-1.84
1,CBA Put,$170.00,-0.00,0.01,-0.00
2,WES Barrier,$18.35,10.62,156.12,117.19
3,Basket Call,$0.19,0.08,NaN,-2.19


In [12]:
# Cell 6: Compute delta‐hedge quantities
df_fmt["Hedge Qty"] = -df_fmt["Delta"]
df_fmt


,Instrument,Price,Delta,Vega,Theta,Hedge Qty
0,BHP Call,$7.86,0.68,21.78,-1.84,-0.68
1,CBA Put,$170.00,-0.00,0.01,-0.00,0.00
2,WES Barrier,$18.35,10.62,156.12,117.19,-10.62
3,Basket Call,$0.19,0.08,NaN,-2.19,-0.08


In [13]:
# Cell 7: Dynamic narrative summary (renders as Markdown)

from IPython.display import Markdown, display

# Build the narrative using the formatted DataFrame (df_fmt)
text = f"""
## Portfolio Hedge Summary

- **BHP European Call**  
  Price = **{df_fmt.loc[0,'Price']}**, Δ = {df_fmt.loc[0,'Delta']}, Vega = {df_fmt.loc[0,'Vega']}, Theta = {df_fmt.loc[0,'Theta']}  
  → Hedge by **selling** {abs(df_fmt.loc[0,'Hedge Qty'])} shares of BHP per call.

- **CBA American Put**  
  Price = **{df_fmt.loc[1,'Price']}**, Δ = {df_fmt.loc[1,'Delta']}, Vega = {df_fmt.loc[1,'Vega']}, Theta = {df_fmt.loc[1,'Theta']}  
  → Hedge by **buying** {abs(df_fmt.loc[1,'Hedge Qty'])} shares of CBA per put.

- **WES Up-and-In Barrier Call**  
  Price = **{df_fmt.loc[2,'Price']}**, Δ = {df_fmt.loc[2,'Delta']}, Vega = {df_fmt.loc[2,'Vega']}, Theta = {df_fmt.loc[2,'Theta']}  
  → Hedge by **selling** {abs(df_fmt.loc[2,'Hedge Qty'])} shares of WES per barrier call.

- **Four-Stock Basket Call**  
  Price = **{df_fmt.loc[3,'Price']}**, Δ = {df_fmt.loc[3,'Delta']}, Vega = {df_fmt.loc[3,'Vega']}, Theta = {df_fmt.loc[3,'Theta']}  
  → Hedge basket by trading each underlying:  
    - Sell 10%×Δ → {abs(0.10 * df_fmt.loc[3,'Hedge Qty']):.2f} shares of BHP  
    - Sell 35%×Δ → {abs(0.35 * df_fmt.loc[3,'Hedge Qty']):.2f} shares of CSL  
    - Sell 15%×Δ → {abs(0.15 * df_fmt.loc[3,'Hedge Qty']):.2f} shares of WDS  
    - Sell 40%×Δ → {abs(0.40 * df_fmt.loc[3,'Hedge Qty']):.2f} shares of MQG  
"""

display(Markdown(text))



## Portfolio Hedge Summary

- **BHP European Call**  
  Price = **$7.86**, Δ = 0.68, Vega = 21.78, Theta = -1.84  
  → Hedge by **selling** 0.68 shares of BHP per call.

- **CBA American Put**  
  Price = **$170.00**, Δ = -0.0, Vega = 0.01, Theta = -0.0  
  → Hedge by **buying** 0.0 shares of CBA per put.

- **WES Up-and-In Barrier Call**  
  Price = **$18.35**, Δ = 10.62, Vega = 156.12, Theta = 117.19  
  → Hedge by **selling** 10.62 shares of WES per barrier call.

- **Four-Stock Basket Call**  
  Price = **$0.19**, Δ = 0.08, Vega = nan, Theta = -2.19  
  → Hedge basket by trading each underlying:  
    - Sell 10%×Δ → 0.01 shares of BHP  
    - Sell 35%×Δ → 0.03 shares of CSL  
    - Sell 15%×Δ → 0.01 shares of WDS  
    - Sell 40%×Δ → 0.03 shares of MQG  
